In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.univariate_selection module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
import pandas as pd

In [3]:
df1 = pd.read_csv('./Amazon_Unlocked_Mobile.csv')

In [4]:
df1

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
...,...,...,...,...,...,...
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [5]:
data_file = './/Amazon_Unlocked_Mobile.csv'
n = 413000  
s = 3000
skip = sorted(random.sample(range(1,n),n-s))


df = pd.read_csv( data_file, delimiter = ",", skiprows = skip)
#41만개의 데이터 중 3000개의 데이터만 뽑아서 분석을 진행 

In [6]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,3,"a pretty capable, durable texting phone, unloc...",2.0
1,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,5,Keeping in mind that this is a sub $100 phone ...,16.0
2,"4 Inch Touch Screen Cell Phone Unlocked, Andro...",NaN,23.90,2,This phone worked well for phone calls. Very s...,2.0
3,"4 Inch Touch Screen Cell Phone Unlocked, Andro...",NaN,23.90,5,Excellent..!!!! Thanks.,0.0
4,"5.5"" Cellphone Unlocked Android 5.1 Quad Core ...",JUNING,99.99,1,Does not work at all. Let the seller checked a...,0.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [7]:
df.isnull().sum()

Product Name      0
Brand Name      492
Price            33
Rating            0
Reviews           1
Review Votes    116
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [10]:
df
#결측치들을 제거하니 41만개의 데이터에서 33만개의 데이터로 축소되었음
#아마 하나의 row에 결측치가 한개라도 존재하면 해당 row를 제거하는 방법을 수행했을것으로 예상 

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,3,"a pretty capable, durable texting phone, unloc...",2.0
1,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,5,Keeping in mind that this is a sub $100 phone ...,16.0
4,"5.5"" Cellphone Unlocked Android 5.1 Quad Core ...",JUNING,99.99,1,Does not work at all. Let the seller checked a...,0.0
5,"5.5"" Unlocked GSM Phones Android 5.1 MTK6580 Q...",JUNING,72.99,5,Wow!!!😱It the best phone ever. I got a case an...,0.0
6,"5.5"" Unlocked Smartphone Dual Sim Quad Core-JU...",JUNING,99.99,2,I ordered two of these based on the reviews..h...,2.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


### Rating Mapping

In [11]:
Review_mapping =  {0: '0',1 : '0', 2 :'0',3 : '0', 4 : '1', 5 : '1'}
Review_mapping
#0~3점까지는 0으로 4~5점까지는 1로 맵핑 

{0: '0', 1: '0', 2: '0', 3: '0', 4: '1', 5: '1'}

In [12]:
Rating = lambda x: Review_mapping.get(x,x)
Rating

<function __main__.<lambda>(x)>

In [13]:
df.Rating.map

<bound method Series.map of 0       3
1       5
4       1
5       5
6       2
       ..
3835    5
3836    3
3837    5
3838    3
3839    4
Name: Rating, Length: 3227, dtype: int64>

In [14]:
Rating = lambda x: Review_mapping.get(x,x)
df['Rating']=df.Rating.map(Rating)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
df['Rating'].unique()

array(['0', '1'], dtype=object)

In [16]:
df
#rating이 잘변환된것을 확인할 수 있다.

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,0,"a pretty capable, durable texting phone, unloc...",2.0
1,"(LANDVO) 5.0"" Capacitive Touch MTK6582 Quad Co...",HTM,69.99,1,Keeping in mind that this is a sub $100 phone ...,16.0
4,"5.5"" Cellphone Unlocked Android 5.1 Quad Core ...",JUNING,99.99,0,Does not work at all. Let the seller checked a...,0.0
5,"5.5"" Unlocked GSM Phones Android 5.1 MTK6580 Q...",JUNING,72.99,1,Wow!!!😱It the best phone ever. I got a case an...,0.0
6,"5.5"" Unlocked Smartphone Dual Sim Quad Core-JU...",JUNING,99.99,0,I ordered two of these based on the reviews..h...,2.0
...,...,...,...,...,...,...
3835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,another great deal great price,0.0
3836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,Ok,0.0
3837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,Passes every drop test onto porcelain tile!,0.0
3838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,I returned it because it did not meet my needs...,0.0


In [17]:
df.to_csv("Sentiment Analysis Final .csv")

### 텍스트 데이터 분리

In [18]:
df.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [19]:
a = df.iloc[:,3]
print(a)
# a는 평점
a.astype('int64')
#맵핑을 한 rating이 object로 되어있어 int로 변환을 하였음 

0       0
1       1
4       0
5       1
6       0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3227, dtype: object


0       0
1       1
4       0
5       1
6       0
       ..
3835    1
3836    0
3837    1
3838    0
3839    1
Name: Rating, Length: 3227, dtype: int64

In [20]:
b = df.iloc[:,4]
# b는 리뷰데이터(텍스트)

In [21]:
doc = pd.concat([a,b], axis = 1)
#기존의 데이터에서 rating과 review로 구성된 새로운 데이터셋을 구성 

### Text Cleaning

In [22]:
def review_to_wordlist(review, remove_stopwords = True):

    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)


In [23]:
import nltk

In [24]:
!pip install nltk.download('stopwords')

/bin/sh: -c: line 0: syntax error near unexpected token `('
/bin/sh: -c: line 0: `pip install nltk.download('stopwords')'


In [25]:
clean_reviews = []
for review in doc['Reviews']:
    clean_reviews.append( " ".join(review_to_wordlist(review)))

In [26]:
len(clean_reviews)
#각 row마다 텍스트 전처리가 잘된것을 확인 

3227

In [27]:
clean_reviews = pd.DataFrame(clean_reviews)

In [28]:
clean_reviews.columns = ['Reviews']
clean_reviews
#컬럼명을 Reviews로 설정 

,Reviews
0,pretti capabl durabl text phone unlock adverti...
1,keep mind sub phone general advertis cheapest ...
2,work let seller check avert phone shop
3,wow best phone ever got case screen protector ...
4,order two base review howev disappoint work ad...
...,...
3222,anoth great deal great price
3223,ok
3224,pass everi drop test onto porcelain tile
3225,return meet need seem good select other


In [29]:
df = pd.concat([clean_reviews, doc['Rating']], axis = 1)
df.isnull().sum()
#무작정 concat을 사용했기때문에 결측값이 발생 
#index를 맞춰주지않아 문제가 발생한 것으로 보인다.

Reviews    599
Rating     599
dtype: int64

In [30]:
clean_reviews.reset_index(drop=True , inplace=True)
clean_reviews
#인덱스를 초기화해줌 

,Reviews
0,pretti capabl durabl text phone unlock adverti...
1,keep mind sub phone general advertis cheapest ...
2,work let seller check avert phone shop
3,wow best phone ever got case screen protector ...
4,order two base review howev disappoint work ad...
...,...
3222,anoth great deal great price
3223,ok
3224,pass everi drop test onto porcelain tile
3225,return meet need seem good select other


In [31]:
b = doc['Rating']

In [32]:
b.reset_index(drop = True, inplace = True)
b
#rating의 인덱스를 초기화해줌 

0       0
1       1
2       0
3       1
4       0
       ..
3222    1
3223    0
3224    1
3225    0
3226    1
Name: Rating, Length: 3227, dtype: object

In [33]:
Data = pd.concat([clean_reviews, doc['Rating']], axis = 1)
Data.isnull().sum()
#인덱스를 초기화하여 각각의 데이터프레임을 결합을 하였으며 결측값이 발생하지 않음 
#인덱스를 초기화해주지않아서 문제가 발생을 하였다

Reviews    0
Rating     0
dtype: int64

In [34]:
Data

,Reviews,Rating
0,pretti capabl durabl text phone unlock adverti...,0
1,keep mind sub phone general advertis cheapest ...,1
2,work let seller check avert phone shop,0
3,wow best phone ever got case screen protector ...,1
4,order two base review howev disappoint work ad...,0
...,...,...
3222,anoth great deal great price,1
3223,ok,0
3224,pass everi drop test onto porcelain tile,1
3225,return meet need seem good select other,0


In [35]:
train_docs, test_docs = train_test_split(Data, test_size = 0.3)
#학습과 테스트를 위해 원본 데이터를 7:3비율로 분할 

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

In [84]:
validation_ratio = math.floor(len(Data['Reviews']) * 0.3)
#validation비율을 나타냄 
max_words = 1500
#최대 단어를 만개
maxlen = 200

In [86]:
tokenizer = Tokenizer(num_words = max_words)
#max_words로 지정한만큼 토큰화 
tokenizer.fit_on_texts(Data['Reviews'])
#texts를 토큰화해줌 
word_index = tokenizer.word_index

In [87]:
data = tokenizer.texts_to_sequences(Data['Reviews'])
print(data)

[[122, 547, 400, 50, 1, 35, 207, 172, 45, 271, 548, 1477, 137, 188, 1062, 321, 7], [113, 643, 1, 745, 207, 983, 821, 1, 549, 111, 3, 1, 984, 46, 619, 93, 14, 265, 549, 56, 164, 822, 166, 1063, 296, 285, 499, 1063, 23, 1346, 90, 322, 509, 51, 22, 194, 869, 1, 16, 372, 10, 60, 39, 870, 1232, 27, 1, 1, 16, 591, 13, 1478, 421, 39, 1064, 10, 10, 16, 372, 322, 164, 87, 51, 746, 7, 1, 823, 225, 468, 434, 1479, 9, 122, 985, 824, 1144, 341, 10, 60, 39, 3, 780, 98, 32, 780, 4, 297, 30, 6, 32, 510, 4, 12, 550, 138, 298, 591, 53, 103, 39, 226, 1347, 551, 43, 1, 511, 104, 118, 1480, 345, 126, 360, 1233, 235, 194, 119, 74, 184, 781, 184, 1234, 410, 1348, 666, 9, 469, 644, 97, 782, 490, 469, 9, 217, 97, 361, 106, 691, 470, 9, 142, 1, 1, 70, 9, 381, 1, 548, 6, 147, 266, 1235, 71, 747, 252, 39, 29, 165, 394, 491, 511, 104, 1, 986, 871, 825, 208, 71, 97, 12, 52, 1, 245, 13, 229, 1478, 236, 1349, 97, 115, 1065, 382, 1236, 61, 27, 1, 285, 112, 382, 826, 372, 12, 492, 1350, 33, 1066, 50, 138, 1481, 86, 123

In [88]:
data = pad_sequences(data, maxlen = maxlen)
print(data)

[[   0    0    0 ... 1062  321    7]
 [  97  361  106 ...  254  289   11]
 [   0    0    0 ...  324    1  528]
 ...
 [   0    0    0 ...  142  366  834]
 [   0    0    0 ...    3  635  461]
 [   0    0    0 ...    4  848  224]]


In [95]:
X_train = data[validation_ratio:]
y_train = Data['Rating'][validation_ratio:]
X_test = data[:validation_ratio]
y_test = Data['Rating'][:validation_ratio]

In [96]:
print(X_train.shape)
print(y_train.shape)

(2259, 200)
(2259,)


In [97]:
y_test

0      0
1      1
2      0
3      1
4      0
      ..
963    1
964    0
965    0
966    1
967    1
Name: Rating, Length: 968, dtype: object

In [98]:
print(X_test.shape)
print(y_test.shape)

(968, 200)
(968,)


### TF-IDF(실행 X)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1,2)
TOP_K = 2000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

def ngram_vectorize(texts, labels):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ
    }
    
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    selector = SelectKBest(f_classif, k = min(TOP_K, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    
    transformed_texts = selector.transform(transformed_texts).astype('float32')
    return transformed_texts
#TF -IDF를 변환하기 위한 각각의 파라미터를 조정하였음 

vect_train_data = ngram_vectorize(train_docs['Reviews'], train_docs['Rating'])
vect_train_data

vect_test_data = ngram_vectorize(test_docs['Reviews'], test_docs['Rating'])
vect_test_data

rf_X_train = vect_train_data.toarray()
rf_y_train = (np.array(train_docs['Rating']))
rf_X_test = vect_test_data.toarray()
rf_y_test = (np.array(test_docs['Rating']))

### Random Forest

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2020)
forest 
#데이터가 너무 커서 최적의 파라미터들을 찾기보다는 default로 두는 것이 가장 좋은 선택이라는 판단 

RandomForestClassifier(n_jobs=-1, random_state=2020)

In [47]:
forest = forest.fit(rf_X_train, rf_y_train)

In [48]:
result = forest.predict(rf_X_test)

In [49]:
rf_y_test = rf_y_test.astype('int64')

In [50]:
result = result.astype('int64')

In [51]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
    #Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [52]:
get_clf_eval(rf_y_test, result)
#랜덤포레스트모델에서는 재현율은 아주 좋은 성능을 보여주며 f1 score는 재현율에 영향을 받기때문에 높은 값으로 나온것롤 보인다 
#하지만 다른 성능지표에서는 그렇게 좋지 못한 성능을 보여주고 있다.

NameError: name 'precision_score' is not defined

### XGBOOST

In [57]:
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [58]:
dtrain = xgb.DMatrix(data = rf_X_train, label = rf_y_train)
dtest = xgb.DMatrix(data= rf_X_test, label = rf_y_test)

#numpy형태의 학습 데이터셋과 테스트 데이터셋을 DMatirx로 변환
#xgboost 모델을 학습시키기위해서는 Dmatrix로 변환을 해야하는것 같음 

In [59]:
params = {'max_depth' : 4,
          'eta' : 0.1,
          'objective' : 'binary:logistic',
          'eval_metric' : 'logloss',
          'early_stoppings' : 30}

num_rounds = 100
#eta : 학습률 
#손실함수 평가지표 : logloss
#파라미터를 

In [60]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params = params, 
                      dtrain = dtrain, 
                      num_boost_round = num_rounds,
                      evals = wlist)

[02:31:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66393	eval-logloss:0.68535
[1]	train-logloss:0.64042	eval-logloss:0.67548
[2]	train-logloss:0.61965	eval-logloss:0.67075
[3]	train-logloss:0.60259	eval-logloss:0.66258
[4]	train-logloss:0.58786	eval-logloss:0.65692
[5]	train-logloss:0.57514	eval-logloss:0.65330
[6]	train-logloss:0.56297	eval-logloss:0.65167
[7]	train-logloss:0.55297	eval-logloss:0.64598
[8]	train-logloss:0.54235	eval-logloss:0.64634
[9]	train-logloss:0.53317	eval-logloss:0.64701
[10]	train-logloss:0.52464	eval-logloss:0.64136
[11]	train-logloss:0.51717	eval-logloss:0.64225
[12]	train-logloss:0.50987	eval-logloss:0.64337
[13]	train-logloss:0.

In [61]:
pred_probs = xgb_model.predict(dtest)
#예측값을 확률로 출력 

In [62]:
preds = [1 if x > 0.4 else 0 for x in pred_probs]

In [63]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
#Confusion Matrix, Accuracy, precision, recall, F1, AUC등을 한번에 보여주는 함수를 생성 

In [64]:
get_clf_eval(rf_y_test, preds)
#재현율에서는 아주 높은 값을 보여주고 있지만 나머지값들에 대해서는 좋지 못한 결과를 보여준다
#또한 AUC가 0.5정도면 좋지 못한 모델이라는 것을 보여준다

오차행렬:
 [[ 24 282]
 [ 15 648]]

정확도: 0.6935
정밀도: 0.6968
재현율: 0.9774
F1: 0.8136
AUC: 0.5279


### LSTM

In [101]:
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

In [52]:
print(X_train.shape)
print(y_train.shape)

(2258, 2000)
(2258,)


In [44]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [45]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Activation, SimpleRNN

embed_size = 64
max_features = 1500


model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#6개의 layer로 구성되어 있으며 성능을 좋게 내기 위해 학습 모델 코드를 참조하였음
#이거는 모델이 잘못된듯 

In [105]:
embedding_dim = 50
max_features = 1500
model = models.Sequential()
model.add(layers.Embedding(max_features, embed_size))
#input node가 만개, output이 50개 각 노드의 길이는 200
model.add(layers.SimpleRNN(50))
model.add(layers.Dense(30, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 64)          96000     
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 50)                5750      
_________________________________________________________________
dense_18 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 31        
Total params: 103,311
Trainable params: 103,311
Non-trainable params: 0
_________________________________________________________________


In [106]:
batch_size = 256
epochs = 5
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.3)
#Validation set을 train셋에서 30%를 두고 학습을 진행 

Epoch 1/5
7/7 [==============================] - 1s 125ms/step - loss: 0.6581 - accuracy: 0.6363 - val_loss: 0.5918 - val_accuracy: 0.7271
Epoch 2/5
7/7 [==============================] - 1s 96ms/step - loss: 0.5954 - accuracy: 0.6863 - val_loss: 0.6237 - val_accuracy: 0.6873
Epoch 3/5
7/7 [==============================] - 1s 82ms/step - loss: 0.5514 - accuracy: 0.7748 - val_loss: 0.5855 - val_accuracy: 0.7389
Epoch 4/5
7/7 [==============================] - 1s 84ms/step - loss: 0.4972 - accuracy: 0.8210 - val_loss: 0.5851 - val_accuracy: 0.7257
Epoch 5/5
7/7 [==============================] - 1s 82ms/step - loss: 0.4307 - accuracy: 0.8703 - val_loss: 0.5680 - val_accuracy: 0.7153


In [107]:
score, acc =model.evaluate(X_test, y_test, batch_size = batch_size)

4/4 [==============================] - 0s 40ms/step - loss: 0.4878 - accuracy: 0.7944


In [108]:
print('Test Score:' , score)
print('Test Accuracy:' , acc)
#69.18%의 정확도를 보여주고 있으며 좋지못한 성능을 보여준다 
#추후 성능을 올릴방법을 찾아서 성능을 올릴예정

Test Score: 0.48776012659072876
Test Accuracy: 0.7944214940071106
